In [2]:
#from nytimesarticle import articleAPI
#api = articleAPI('a0HA3uBISDkGyvUGR3FeoAGybtDVPPM5')

In [4]:
#articles = api.search( q = 'immigration', 
#     begin_date = 20120101, end_date =20120201 )

# create TIDF on an example call from R

https://towardsdatascience.com/latent-semantic-analysis-sentiment-classification-with-python-5f657346f6a3

In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
import nltk
import gensim 
from gensim.models import Word2Vec 
from gensim.models.wrappers import FastText 

from gensim.models import FastText


from PyDictionary import PyDictionary # utilize this library in order to do webscrapping
from IPython.display import clear_output, display

from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
import nltk 
import pandas as pd
from pandas import *
import pandasql
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
import re
from sklearn.feature_extraction import text
from builtins import input
from statistics import mean



from statistics import mode, StatisticsError

import functools


import re
from sklearn.feature_extraction import text
global cases

from pandas import ExcelWriter
import xlsxwriter

import xlwt
from xlwt.Workbook import *


from tqdm import tqdm_notebook
from nltk.corpus import wordnet

from statistics import *

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import pandas as pd
import re

In [15]:
df = pd.read_csv('Sentiment-Analysis-NYT-Articlesdir\\allNYTSearch1980to2020.txt', sep = "\t", encoding = 'iso-8859-1')

In [19]:
df.columns

Index(['status', 'copyright', 'response.docs.abstract',
       'response.docs.web_url', 'response.docs.snippet',
       'response.docs.lead_paragraph', 'response.docs.print_section',
       'response.docs.print_page', 'response.docs.source',
       'response.docs.pub_date', 'response.docs.document_type',
       'response.docs.news_desk', 'response.docs.section_name',
       'response.docs.type_of_material', 'response.docs._id',
       'response.docs.word_count', 'response.docs.uri',
       'response.docs.headline.main', 'response.docs.headline.kicker',
       'response.docs.headline.content_kicker',
       'response.docs.headline.print_headline', 'response.docs.headline.name',
       'response.docs.headline.seo', 'response.docs.headline.sub',
       'response.docs.byline.original', 'response.docs.byline.organization',
       'response.meta.hits', 'response.meta.offset', 'response.meta.time',
       'response.docs.subsection_name'],
      dtype='object')

Preprocess evertyhing in python and not R: migrating code

In [20]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    #tokenizer = RegexpTokenizer(r'\w+')
    #tokens = tokenizer.tokenize(rem_num)
    tokens = nltk.tokenize.word_tokenize(rem_num) 
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    #stem_words=[stemmer.stem(w) for w in filtered_words]
    #lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)
    




In [87]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

prohibitedWords = []

def preprocess_regex(row):
    row=str(row)
    row = row.lower()
    row=row.replace('-'," ") 
    row=row.replace(','," ")
    row=row.replace('.'," ")
    row=row.replace('+'," ")
    row=row.replace('"'," ")
    row=row.replace("''","")
    big_regex = re.compile(r'\b%s\b' % r'\b|\b'.join(map(re.escape, prohibitedWords)))  #this would replace 'random' in 'random words' but not in 'pseudorandom words'
    cleantext = re.sub(big_regex, '', row)
    rem_num = re.sub('[0-9]+', '', cleantext)
    #tokenizer = RegexpTokenizer(r'\w+')  #this yields lower counts of words
    tokens = nltk.tokenize.word_tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')] #if it appears more than two times get rid off the stopwords, change this
    return " ".join(tokens)


In [21]:
df['cleanText']=df['response.docs.lead_paragraph'].map(lambda s:preprocess(s)) 

Tokenized_lead_paragraphs = df.apply(lambda row: nltk.word_tokenize(row['cleanText']), axis=1)

In [50]:
def tokenizing_words_in_product_title(df, col):    
    pt_clean = df[col].str.cat(sep=' ')
    words = nltk.tokenize.word_tokenize(pt_clean)
    word_dist = nltk.FreqDist(words)
    top_N = len(words)

    #creating a dataframe of these words plus id so later we can merge.
    C_UW_F= pd.DataFrame(word_dist.most_common(top_N),  #C_UW_F = category unique word final
                    columns=['Word', 'Frequency'])
    C_UW_F["id"] = C_UW_F.index + 1
    return (C_UW_F)
#C_UW_F



In [51]:
U_words_corpus=tokenizing_words_in_product_title(df, 'cleanText')

#U_words_corpus


In [38]:
list_query = U_words_corpus['Word'] #words and its meanings

meanings_for_words_in_product_title = [] 
for word in list_query: # Your word list
    ss = wordnet.synsets(word)
    if ss:
        definition = ss[0].definition() 
        meanings_for_words_in_product_title.append(definition)
    else:
        word = None
        meanings_for_words_in_product_title.append(word)

In [56]:
#meanings_for_words_in_product_title

In [66]:
df_unqiue_words = pd.DataFrame({'word':list_query, 'tag':meanings_for_words_in_product_title})

df_unqiue_words['counts_of_words'] = U_words_corpus['Frequency']

len(df_unqiue_words)

8709

# Senitment analysis

In [22]:
df['cleanText'][1]

'michael wright caroline rand herron last week pedro rodriguez must wondered whether boat took wrong turn last april cuban refugee boatlift shortly arrived united states trundled federal penitentiary leavenworth kan'

In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [24]:
def sentiment_analyzer_scores(sentence):
    scores = analyser.polarity_scores(sentence)
    #score = ("{:-<40} {}".format(sentence, str(string_words)))
    return scores

In [25]:
clean_text=list(df['cleanText'])

sentiment_scores = []
for i in range(len(clean_text)):
    sentiment_scores.append(sentiment_analyzer_scores(clean_text[i]))

In [26]:
df['sentiment_scores'] = sentiment_scores

In [27]:
len(df)

21629

In [25]:
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize, RegexpTokenizer


def get_word_sentiment(text):
    neg_word_list = []
    pos_word_list = []
    neu_word_list = []
    
    tokenized_text = nltk.word_tokenize(text)
    
    for word in tokenized_text:
        if (analyser.polarity_scores(word)['compound']) >= 0.1:
            pos_word_list.append(word)
        elif (analyser.polarity_scores(word)['compound']) <= -0.1:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)
    return {'Positive':pos_word_list,'Neutral': neu_word_list,'Negative':neg_word_list}
    #return pos_word_list,neu_word_list,neg_word_list


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pdgac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pdgac\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [34]:
lists_sentiment_P_N_N = []
for i in range(len(df)):
    lists_sentiment_P_N_N.append(get_word_sentiment(clean_text[i]))

In [36]:
len(lists_sentiment_P_N_N)

1459

In [1]:
lists_sentiment_P_N_N[1]

NameError: name 'lists_sentiment_P_N_N' is not defined

# TF-IDF

TF (Term Frequency) measures the frequency of a word in a document.

TF = (Number of time the word occurs in the text) / (Total number of words in text)

IDF (Inverse Document Frequency) measures the rank of the specific word for its relevancy within the text. Stop words which contain unnecessary information such as “a”, “into” and “and” carry less importance in spite of their occurrence.

IDF = (Total number of documents / Number of documents with word t in it)

Thus, the TF-IDF is the product of TF and IDF:

TF-IDF = TF * IDF

https://iyzico.engineering/how-to-calculate-tf-idf-term-frequency-inverse-document-frequency-from-the-beatles-biography-in-c4c3cd968296

In [51]:
cvec = CountVectorizer( min_df=3, max_df=0.5, ngram_range=(1,2))
sf = cvec.fit_transform(df['cleanText'])

In [52]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(sf)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})

In [54]:
weights_df.sort_values(by='weight', ascending=False).head(50)

,term,weight
1847,migrants,0.021299
367,border,0.018927
3143,trump,0.018896
3209,united,0.018171
2285,president,0.017406
2850,states,0.016919
3211,united states,0.016074
1446,immigration,0.015551
1823,mexico,0.015008
1993,new,0.014969


# Word2vec models 

In [56]:
model_by_category = gensim.models.Word2Vec(Tokenized_lead_paragraphs, min_count = 10,  
                               window = 5, sg = 1) #CBOW method, the idea is that given a context, we want to know which word is most likely to appear in it

vocabulary = model_by_category.wv.vocab
#print(vocabulary)


sim_words = model_by_category.wv.most_similar('central', topn = 10)
print(sim_words)

[('caravan', 0.9978567361831665), ('mexico', 0.9973210692405701), ('crossing', 0.9973007440567017), ('american', 0.9972114562988281), ('southern', 0.9970765113830566), ('america', 0.9969837665557861), ('surge', 0.9969432353973389), ('northern', 0.9967454075813293), ('guatemala', 0.9966573715209961), ('san', 0.9965139031410217)]


In [64]:
model_most_similar_words_lead_P = FastText(Tokenized_lead_paragraphs, size=100, window=5, min_count=5)

In [65]:
model_most_similar_words_lead_P.wv.most_similar("migration", topn = 50)

[('immigration', 0.9999980926513672),
 ('generation', 0.9999960660934448),
 ('nation', 0.9999956488609314),
 ('separation', 0.9999954700469971),
 ('deportation', 0.9999953508377075),
 ('coalition', 0.9999949932098389),
 ('detention', 0.9999946355819702),
 ('population', 0.9999945163726807),
 ('destination', 0.9999943971633911),
 ('station', 0.999994158744812),
 ('action', 0.9999940395355225),
 ('attention', 0.9999939203262329),
 ('edition', 0.9999937415122986),
 ('nomination', 0.9999936819076538),
 ('situation', 0.9999935626983643),
 ('nations', 0.9999933838844299),
 ('deportations', 0.9999932646751404),
 ('investigation', 0.9999927878379822),
 ('legislation', 0.9999927282333374),
 ('nationwide', 0.9999927282333374),
 ('question', 0.9999924302101135),
 ('election', 0.9999923706054688),
 ('administration', 0.9999923706054688),
 ('conditions', 0.9999923706054688),
 ('national', 0.9999922513961792),
 ('organization', 0.9999922513961792),
 ('opposition', 0.9999921321868896),
 ('protection'